In [1]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd

import torch
import torchtext

import matplotlib.pyplot as plt
from tqdm import tqdm

import cfg

from text_classification import trainutils
from text_classification.layers import *
from text_classification.logger import logger
from text_classification.datautils import *
from text_classification.trainutils import get_metrics

from train import train

%load_ext autoreload
%autoreload 2

07-18 13:46 summarizer.preprocessing.cleaner INFO     'pattern' package not found; tag filters are not available for English


In [7]:
%reload_ext autoreload

## CharCNN

In [2]:
text_field = torchtext.data.Field(
    lower=True, include_lengths=False, tensor_type=torch.FloatTensor, batch_first=True,
    tokenize=lambda x: x, use_vocab=False, sequential=False
)
label_field = torchtext.data.Field(sequential=False, use_vocab=False)

CharIMDB.maxlen = 512
train_data, test_data = CharIMDB.splits(text_field, label_field, root='../.data')

train_dataloader, val_dataloader, test_dataloader = \
    trainutils.get_dataloaders(train_data, test_data, batch_size=cfg.train.batch_size,
                               valid_size=cfg.train.val_size)


In [12]:
model = CharCNN(128, 5, 512, len(cfg.alphabet))

for lr in [1e0, 1e-1, 1e-2, 1e-3, 1e-4]:
    trained_model, results = \
        train(model,
              train_dataloader,
              val_dataloader,
              test_dataloader,
              test_dataloader,
              lr=lr,
              noise_level=0,
              comment='',
              save_model_path=None,
              save_results_path=None)


07-04 18:06 root         INFO     Writer: runs/Jul04_18-06-09_lyalin_CharCNN_lr0_dropout0.5_noise_level0.0000
07-04 18:06 root         INFO     Epoch 0. Global step 665. T=0.17min
07-04 18:06 root         INFO     In-batch loss      : 0.6937
07-04 18:06 root         INFO     Training accuracy  : 0.5008, f1: 0.6674
07-04 18:06 root         INFO     Validation accuracy: 0.4955, f1: 0.6626
/home/not_a_robot/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
07-04 18:07 root         INFO     Epoch 9. Global step 6650. T=1.65min
07-04 18:07 root         INFO     In-batch loss      : 0.6934
07-04 18:07 root         INFO     Training accuracy  : 0.4992, f1: 0.0000
07-04 18:07 root         INFO     Validation accuracy: 0.5045, f1: 0.0000
07-04 18:07 root         INFO     Calculating test metrics... Time T=1.65min
100%|██████████

In [ ]:
best_f1 = 0
best_model = None
best_params = None

results = []


for _ in tqdm(range(150)):
    n_filters = int(np.random.choice([16, 32, 64, 128, 256, 512, 1024]))
    cnn_kernel_size = int(np.random.choice([3, 5, 7]))
#     maxlen = int(np.random.choice([256, 512, 1024]))
    maxlen = 512
    dropout = np.random.rand()
    
    params = {
        'n_filters': n_filters,
        'cnn_kernel_size': cnn_kernel_size,
        'maxlen': maxlen,
        'dropout': dropout
    }
    
    logger.info(params)

    try:
        model = CharCNN(alphabet_len=len(cfg.alphabet), **params)

        trained_model = \
            train(model,
                  train_dataloader,
                  val_dataloader,
                  noise_level=0,
                  lr=lr,
                  comment='hyperparameters_search',
                  save_model_path=None)
        metrics = get_metrics(trained_model, val_dataloader)

        if metrics['f1'] > best_f1:
            best_f1 = metrics['f1']
            best_model = model
            best_params = params

        params.update(metrics)
        results.append(params)
        pd.DataFrame(results).to_csv('../results/CharCNN_hyperparams.csv')
    except Exception as e:
        logger.error(e)


In [28]:
best_f1, best_params

(0.7391752577319587,
 {'n_filters': 1024,
  'cnn_kernel_size': 5,
  'maxlen': 512,
  'dropout': 0.09851507346853383,
  'accuracy': 0.7301333333333333,
  'f1': 0.7391752577319587})

In [ ]:
best_f1 = 0
best_model = None
best_params = None

results = []


for _ in tqdm(range(150)):
    n_filters = int(np.random.choice([16, 32, 64, 128, 256, 512, 1024]))
    cnn_kernel_size = int(np.random.choice([3, 5, 7]))
#     maxlen = int(np.random.choice([256, 512, 1024]))
    maxlen = 512
    dropout = np.random.rand() * 0.5 + 0.5
    
    params = {
        'n_filters': n_filters,
        'cnn_kernel_size': cnn_kernel_size,
        'maxlen': maxlen,
        'dropout': dropout
    }
    
    logger.info(params)

    try:
        model = CharCNN(alphabet_len=len(cfg.alphabet), **params)

        trained_model = \
            train(model,
                  train_dataloader,
                  val_dataloader,
                  noise_level=0,
                  lr=lr,
                  comment='hyperparameters_search',
                  save_model_path=None)
        metrics = get_metrics(trained_model, val_dataloader)

        if metrics['f1'] > best_f1:
            best_f1 = metrics['f1']
            best_model = model
            best_params = params

        params.update(metrics)
        results.append(params)
        pd.DataFrame(results).to_csv('../results/CharCNN_hyperparams.csv')
    except Exception as e:
        logger.error(e)






  0%|          | 0/150 [00:00<?, ?it/s]07-05 18:09 root         INFO     {'n_filters': 16, 'cnn_kernel_size': 3, 'maxlen': 512, 'dropout': 0.7587327768268632}
07-05 18:09 root         INFO     Writer: runs/Jul05_18-09-08_lyalin_CharCNN_lr4_dropout0.7587327768268632_noise_level0.0000hyperparameters_search
07-05 18:09 root         INFO     Epoch 0. Global step 665. T=0.16min
07-05 18:09 root         INFO     In-batch loss      : 0.7718
07-05 18:09 root         INFO     Training accuracy  : 0.5236, f1: 0.6120
07-05 18:09 root         INFO     Validation accuracy: 0.5096, f1: 0.5986


In [31]:
best_f1, best_params

(0.7124064998710343,
 {'n_filters': 1024,
  'cnn_kernel_size': 7,
  'maxlen': 512,
  'dropout': 0.6716150874837757,
  'accuracy': 0.7026666666666667,
  'f1': 0.7124064998710343})

In [ ]:
for _ in tqdm(range(150)):
    lr = np.random.uniform()
    n_filters = int(np.random.choice([16, 32, 64, 128, 256, 512, 1024]))
    cnn_kernel_size = int(np.random.choice([3, 5, 7]))
#     maxlen = int(np.random.choice([256, 512, 1024]))
    maxlen = 512
    dropout = np.random.rand() * 0.5 + 0.5

    params = {
        'n_filters': n_filters,
        'cnn_kernel_size': cnn_kernel_size,
        'maxlen': maxlen,
        'dropout': dropout
    }
    
    logger.info(params)

    try:
        model = CharCNN(alphabet_len=len(cfg.alphabet), **params)

        trained_model = \
            train(model,
                  train_dataloader,
                  val_dataloader,
                  noise_level=0,
                  lr=lr,
                  comment='hyperparameters_search',
                  save_model_path=None)
        metrics = get_metrics(trained_model, val_dataloader)

        if metrics['f1'] > best_f1:
            best_f1 = metrics['f1']
            best_model = model
            best_params = params

        params.update(metrics)
        results.append(params)
        pd.DataFrame(results).to_csv('../results/CharCNN_hyperparams.csv')
    except Exception as e:
        logger.error(e)


In [34]:
best_f1, best_params

(0.7124064998710343,
 {'n_filters': 1024,
  'cnn_kernel_size': 7,
  'maxlen': 512,
  'dropout': 0.6716150874837757,
  'accuracy': 0.7026666666666667,
  'f1': 0.7124064998710343})

In [ ]:
lr = 10**np.random.uniform(-4, 0)
n_filters = int(np.random.choice([16, 32, 64, 128, 256, 512, 1024]))
cnn_kernel_size = int(np.random.choice([3, 5, 7]))
maxlen = 512
dropout = np.random.rand() * 0.5 + 0.5

In [8]:
best_f1 = 0.7124064998710343

In [9]:
if 0 > best_f1:
    best_f1 = 0

lr = 0.001
n_filters = 128
cnn_kernel_size = 5
maxlen = 512
dropout = 0.5

params = {
    'n_filters': n_filters,
    'cnn_kernel_size': cnn_kernel_size,
    'maxlen': maxlen,
    'dropout': dropout
}

model = CharCNN(alphabet_len=len(cfg.alphabet), **params)

trained_model = \
    train(model,
          train_dataloader,
          val_dataloader,
          epochs=30,
          noise_level=0,
          lr=lr,
          log_every=2,
          comment='hyperparameters_search',
          save_model_path=None)
metrics = get_metrics(trained_model, val_dataloader)

if metrics['f1'] > best_f1:
    print('YES!')
    best_f1 = metrics['f1']
    best_model = model
    best_params = params
    print(best_params)

07-09 14:12 root         INFO     Writer: runs/Jul09_14-12-47_lyalin_CharCNN_lr3_dropout0.5_noise_level0.0000hyperparameters_search
07-09 14:12 root         INFO     Epoch 0. Global step 665. T=0.16min
07-09 14:12 root         INFO     In-batch loss      : 0.7336
07-09 14:12 root         INFO     Training accuracy  : 0.6725, f1: 0.6801
07-09 14:12 root         INFO     Validation accuracy: 0.6603, f1: 0.6687
07-09 14:13 root         INFO     Epoch 2. Global step 1995. T=0.50min
07-09 14:13 root         INFO     In-batch loss      : 0.2334
07-09 14:13 root         INFO     Training accuracy  : 0.7330, f1: 0.7254
07-09 14:13 root         INFO     Validation accuracy: 0.7120, f1: 0.7005
07-09 14:13 root         INFO     Epoch 4. Global step 3325. T=0.82min
07-09 14:13 root         INFO     In-batch loss      : 0.5750
07-09 14:13 root         INFO     Training accuracy  : 0.7641, f1: 0.7677
07-09 14:13 root         INFO     Validation accuracy: 0.7368, f1: 0.7380
07-09 14:13 root         I

YES!
{'n_filters': 128, 'cnn_kernel_size': 5, 'maxlen': 512, 'dropout': 0.5}


## Added batch norm

In [ ]:
model = CharCNN(128, 5, 512, len(cfg.alphabet))

trained_model, results = \
    train(model,
          train_dataloader,
          val_dataloader,
          lr=1e-3,
          noise_level=0,
          comment='',
          save_model_path=None)


# AttentionedYoonKim

In [9]:
%%time
text_field = torchtext.data.Field(
    lower=True, include_lengths=False, tensor_type=torch.FloatTensor, batch_first=True,
    tokenize='spacy', use_vocab=False
)
label_field = torchtext.data.Field(sequential=False, use_vocab=False)

train_data, test_data = HierarchicalIMDB.splits(text_field, label_field, root='../.data')
train_dataloader, val_dataloader, test_dataloader = \
    trainutils.get_dataloaders(train_data, test_data, batch_size=cfg.train.batch_size,
                               valid_size=cfg.train.val_size)


In [10]:
best_f1 = 0

In [11]:
if 0 > best_f1:
    best_f1 = 0

lr = 0.001
n_filters = 128  # int(np.random.choice([16, 32, 64, 128, 256, 512, 1024]))
cnn_kernel_size = 5
maxlen = 512
dropout = 0.5
hidden_dim_out = 128
embedding_dim = 74

params = {
    'n_filters': n_filters,
    'cnn_kernel_size': cnn_kernel_size,
    'hidden_dim_out': hidden_dim_out,
    'heads': 1,
#     'maxlen': maxlen,
    'embedding_dim': embedding_dim,
    'dropout': dropout,
#     'pool_kernel_size': 4
}

model = AttentionedYoonKimModel(**params)

trained_model = \
    train(model,
          train_dataloader,
          val_dataloader,
          epochs=30,
          noise_level=0,
          lr=lr,
          log_every=2,
          comment='hyperparameters_search',
          save_model_path=None)
metrics = get_metrics(trained_model, val_dataloader)

if metrics['f1'] > best_f1:
    print('YES!')
    best_f1 = metrics['f1']
    best_model = model
    best_params = params
    print(best_params)

/home/not_a_robot/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
07-09 15:21 root         INFO     Writer: runs/Jul09_15-21-51_lyalin_AttentionedYoonKimModel_lr3_dropout0.5_noise_level0.0000hyperparameters_search
07-09 15:23 root         INFO     Epoch 0. Global step 665. T=2.03min
07-09 15:23 root         INFO     In-batch loss      : 0.7096
07-09 15:23 root         INFO     Training accuracy  : 0.5078, f1: 0.4185
07-09 15:23 root         INFO     Validation accuracy: 0.5064, f1: 0.4210
07-09 15:28 root         INFO     Epoch 2. Global step 1995. T=6.32min
07-09 15:28 root         INFO     In-batch loss      : 0.7181
07-09 15:28 root         INFO     Training accuracy  : 0.6406, f1: 0.6223
07-09 15:28 root         INFO     Validation accuracy: 0.6248, f1: 0

KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "

In [12]:
len(train_dataloader), len(val_dataloader)

(665, 118)

In [18]:
lr = 1e-3
n_filters = 64
cnn_kernel_size = 5
dropout = np.random.rand() * 0.9 + 0.1
hidden_dim_out = 256
embedding_dim = int(np.random.randint(32, 128))

params = {
    'n_filters': n_filters,
    'cnn_kernel_size': cnn_kernel_size,
    'hidden_dim_out': hidden_dim_out,
    'heads': 1,
#     'maxlen': maxlen,
    'embedding_dim': embedding_dim,
    'dropout': dropout,
#     'pool_kernel_size': 4
}

logger.info('Parameters: %s' % params)
model = AttentionedYoonKimModel(**params)

trained_model = \
    train(model,
          train_dataloader,
          val_dataloader,
          epochs=30,
          noise_level=0,
          lr=lr,
          log_every=2,
          comment='hyperparameters_search_manual',
          save_model_path=None)
metrics = get_metrics(trained_model, val_dataloader)

if metrics['f1'] > best_f1:
    logger.info('YES!')
    best_f1 = metrics['f1']
    best_model = model
    best_params = params
    print(best_params)

07-09 19:44 root         INFO     Parameters: {'n_filters': 64, 'cnn_kernel_size': 5, 'hidden_dim_out': 256, 'heads': 1, 'embedding_dim': 51, 'dropout': 0.49569537913353157}
/home/not_a_robot/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.49569537913353157 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
07-09 19:44 root         INFO     Writer: runs/Jul09_19-44-56_lyalin_AttentionedYoonKimModel_lr3_dropout0.49569537913353157_noise_level0.0000hyperparameters_search_manual
07-09 19:47 root         INFO     Epoch 0. Global step 665. T=2.07min
07-09 19:47 root         INFO     In-batch loss      : 0.6998
07-09 19:47 root         INFO     Training accuracy  : 0.5013, f1: 0.6622
07-09 19:47 root         INFO     Validation accuracy: 0.5016, f1: 0.6605
07-09 19:51 root         INFO     Epoch 2. Global step 1995. T

KeyboardInterrupt: 

Traceback (most recent call last):
Traceback (most recent call last):
Process Process-1742:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/not_a_robot/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/not_a_robot/.local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = colla

In [ ]:
if 0 > best_f1:
    best_f1 = 0

for _ in range(100):
    lr = 10**np.random.uniform(-4, -3)
    n_filters = int(np.random.choice([32, 64, 128, 256]))
    cnn_kernel_size = int(np.random.choice([3, 5, 7]))
    dropout = np.random.rand() * 0.9 + 0.1
    hidden_dim_out = int(np.random.choice([64, 128, 256]))
    embedding_dim = int(np.random.randint(32, 128))

    params = {
        'n_filters': n_filters,
        'cnn_kernel_size': cnn_kernel_size,
        'hidden_dim_out': hidden_dim_out,
        'heads': 1,
    #     'maxlen': maxlen,
        'embedding_dim': embedding_dim,
        'dropout': dropout,
    #     'pool_kernel_size': 4
    }

    model = AttentionedYoonKimModel(**params)
    params['lr'] = lr
    logger.info('Parameters: %s' % params)

    trained_model = \
        train(model,
              train_dataloader,
              val_dataloader,
              epochs=20,
              noise_level=0,
              lr=lr,
              log_every=2,
              comment='hyperparameters_search_random',
              save_model_path=None)
    metrics = get_metrics(trained_model, val_dataloader)

    if metrics['f1'] > best_f1:
        logger.info('YES!, f1: %s, parameters: %s' % (metrics['f1'], str(params)))
        best_f1 = metrics['f1']
        best_model = model
        best_params = params
        logger.info(best_params)


07-09 20:22 root         INFO     Parameters: {'n_filters': 32, 'cnn_kernel_size': 7, 'hidden_dim_out': 64, 'heads': 1, 'embedding_dim': 67, 'dropout': 0.363531569594306}
/home/not_a_robot/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.363531569594306 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
07-09 20:22 root         INFO     Writer: runs/Jul09_20-22-54_lyalin_AttentionedYoonKimModel_lr3_dropout0.363531569594306_noise_level0.0000hyperparameters_search_random
07-09 20:24 root         INFO     Epoch 0. Global step 665. T=2.03min
07-09 20:24 root         INFO     In-batch loss      : 0.7092
07-09 20:24 root         INFO     Training accuracy  : 0.5140, f1: 0.3765
07-09 20:24 root         INFO     Validation accuracy: 0.5061, f1: 0.3743


In [20]:
best_f1

0.8667728237791931

# YoonKimModel

In [2]:
%%time
text_field = torchtext.data.Field(
    lower=True, include_lengths=False, tensor_type=torch.FloatTensor, batch_first=True,
    tokenize='spacy', use_vocab=False
)
label_field = torchtext.data.Field(sequential=False, use_vocab=False)

train_data, test_data = HierarchicalIMDB.splits(text_field, label_field, root='../.data')
train_dataloader, val_dataloader, test_dataloader = \
    trainutils.get_dataloaders(train_data, test_data, batch_size=cfg.train.batch_size,
                               valid_size=cfg.train.val_size)


CPU times: user 3min 9s, sys: 1.35 s, total: 3min 11s
Wall time: 3min 15s


In [3]:
best_f1 = 0

In [4]:
lr = 1e-3
n_filters = 32
cnn_kernel_size = 5
dropout = 0.5
hidden_dim_out = 64
embedding_dim = int(np.random.randint(32, 128))

params = {
    'n_filters': n_filters,
    'cnn_kernel_size': cnn_kernel_size,
    'hidden_dim_out': hidden_dim_out,
#     'maxlen': maxlen,
    'embedding_dim': embedding_dim,
    'dropout': dropout,
}

model = YoonKimModel(**params)
params['lr'] = lr
logger.info('Parameters: %s' % params)

trained_model = \
    train(model,
          train_dataloader,
          val_dataloader,
          epochs=20,
          noise_level=0,
          lr=lr,
          log_every=2,
          comment='hyperparameters_search_manual',
          save_model_path=None)
metrics = get_metrics(trained_model, val_dataloader)

if metrics['f1'] > best_f1:
    logger.info('YES!, f1: %s, acc: %s, parameters: %s' % (
        metrics['f1'], metrics['acc'], str(params)
    ))
    best_f1 = metrics['f1']
    best_model = model
    best_params = params
    logger.info(best_params)

07-18 13:49 root         INFO     Parameters: {'n_filters': 32, 'cnn_kernel_size': 5, 'hidden_dim_out': 64, 'embedding_dim': 101, 'dropout': 0.5, 'lr': 0.001}
07-18 13:49 root         INFO     Writer: runs/Jul18_13-49-42_lyalin_YoonKimModel_lr3_dropout0.5_noise_level0.0000hyperparameters_search_manual
07-18 13:51 root         INFO     Texts ninth symbols:
 tensor([ 23,  21,   3,   7,   7,  15,   7,  16,  42,  11,   3,   3,
          3,  21,   3,  22,  21,  23,  17,  11,   0,  15,   8,  11,
         20,  21,  27,  17,  20,  14,  17,   0], device='cuda:0')
07-18 13:51 root         INFO     Prediction:
 tensor([[-0.1029, -0.1313],
        [-0.1114, -0.1208],
        [-0.1114, -0.1208],
        [-0.1114, -0.1208],
        [-0.1114, -0.1208],
        [-0.1114, -0.1208],
        [-0.1114, -0.1208],
        [-0.1982, -0.0447],
        [-0.1462, -0.0942],
        [-0.1234, -0.0969],
        [-0.1114, -0.1208],
        [-0.1129, -0.1155],
        [-0.1114, -0.1208],
        [-0.1114, -0.1208],


KeyboardInterrupt: 

In [8]:
lr = 1e-3
n_filters = 512
cnn_kernel_size = 7
dropout = 0.5
hidden_dim_out = 512
embedding_dim = int(np.random.randint(32, 128))

params = {
    'n_filters': n_filters,
    'cnn_kernel_size': cnn_kernel_size,
    'hidden_dim_out': hidden_dim_out,
#     'maxlen': maxlen,
    'embedding_dim': embedding_dim,
    'dropout': dropout,
}

model = YoonKimModel(**params)
params['lr'] = lr
logger.info('Parameters: %s' % params)

trained_model = \
    train(model,
          train_dataloader,
          val_dataloader,
          epochs=20,
          noise_level=0,
          lr=lr,
          log_every=2,
          comment='hyperparameters_search_manual',
          save_model_path=None)
metrics = get_metrics(trained_model, val_dataloader)

if metrics['f1'] > best_f1:
    logger.info('YES!, f1: %s, parameters: %s' % (metrics['f1'], str(params)))
    best_f1 = metrics['f1']
    best_model = model
    best_params = params
    logger.info(best_params)

07-10 12:36 root         INFO     Parameters: {'n_filters': 512, 'cnn_kernel_size': 7, 'hidden_dim_out': 512, 'embedding_dim': 37, 'dropout': 0.5, 'lr': 0.001}
07-10 12:36 root         INFO     Writer: runs/Jul10_12-36-20_lyalin_YoonKimModel_lr3_dropout0.5_noise_level0.0000hyperparameters_search_manual
07-10 12:39 root         INFO     Epoch 0. Global step 665. T=2.85min
07-10 12:39 root         INFO     In-batch loss      : 0.6960
07-10 12:39 root         INFO     Training accuracy  : 0.5008, f1: 0.6674
07-10 12:39 root         INFO     Validation accuracy: 0.4955, f1: 0.6626
07-10 12:44 root         INFO     Epoch 2. Global step 1995. T=8.49min
07-10 12:44 root         INFO     In-batch loss      : 0.7064
07-10 12:44 root         INFO     Training accuracy  : 0.5242, f1: 0.3710
07-10 12:44 root         INFO     Validation accuracy: 0.5139, f1: 0.3588
07-10 12:50 root         INFO     Epoch 4. Global step 3325. T=14.10min
07-10 12:50 root         INFO     In-batch loss      : 0.6994
0

In [4]:
if 0 > best_f1:
    best_f1 = 0

for _ in range(100):
    lr = 10**np.random.uniform(-4, -3)
    n_filters = int(np.random.choice([32, 64, 128, 256]))
    cnn_kernel_size = int(np.random.choice([3, 5, 7]))
    dropout = np.random.rand() * 0.9 + 0.1
    hidden_dim_out = int(np.random.choice([64, 128, 256]))
    embedding_dim = int(np.random.randint(32, 128))

    params = {
        'n_filters': n_filters,
        'cnn_kernel_size': cnn_kernel_size,
        'hidden_dim_out': hidden_dim_out,
    #     'maxlen': maxlen,
        'embedding_dim': embedding_dim,
        'dropout': dropout,
    }

    model = YoonKimModel(**params)
    params['lr'] = lr
    logger.info('Parameters: %s' % params)

    trained_model = \
        train(model,
              train_dataloader,
              val_dataloader,
              epochs=20,
              noise_level=0,
              lr=lr,
              log_every=2,
              comment='hyperparameters_search_random',
              save_model_path=None)
    metrics = get_metrics(trained_model, val_dataloader)

    if metrics['f1'] > best_f1:
        logger.info('YES!, f1: %s, parameters: %s' % (metrics['f1'], str(params)))
        best_f1 = metrics['f1']
        best_model = model
        best_params = params
        logger.info(best_params)

07-11 19:29 root         INFO     Parameters: {'n_filters': 128, 'cnn_kernel_size': 5, 'hidden_dim_out': 64, 'embedding_dim': 103, 'dropout': 0.741404662378827, 'lr': 0.0001287332116002057}


NameError: name 'train_dataloader' is not defined

In [12]:
best_f1, best_params

(0.8641318798191969,
 {'n_filters': 256,
  'cnn_kernel_size': 5,
  'hidden_dim_out': 64,
  'embedding_dim': 72,
  'dropout': 0.6243415666771305,
  'lr': 0.000667854542403568})

## FastText

In [4]:
from gensim.models import FastText

In [5]:
embeddings = FastText.load_fasttext_format(cfg.data.fasttext_path)

07-13 16:18 gensim.models.fasttext INFO     loading 111051 words for fastText model from /data/embeddings/wiki.simple.bin
07-13 16:18 gensim.models.fasttext INFO     loading weights for 111051 words for fastText model from /data/embeddings/wiki.simple.bin
07-13 16:18 gensim.models.fasttext INFO     loaded (111051, 300) weight matrix for fastText model from /data/embeddings/wiki.simple.bin


In [6]:
text_field = torchtext.data.Field(
    lower=True, include_lengths=False, tensor_type=torch.FloatTensor, batch_first=True,
    tokenize='spacy', use_vocab=False
)
label_field = torchtext.data.Field(sequential=False, use_vocab=False)

train_data, test_data = FastTextIMDB.splits(text_field, label_field, embeddings=embeddings, root='../.data')
train_dataloader, val_dataloader, test_dataloader = \
    trainutils.get_dataloaders(train_data, test_data, batch_size=cfg.train.batch_size,
                               valid_size=cfg.train.val_size)


Got embeddings
Got embeddings


In [7]:
len(train_dataloader), len(val_dataloader)

(665, 118)

In [8]:
best_f1 = 0

In [14]:
if 0 > best_f1:
    best_f1 = 0

for _ in range(1000):
    lr = 10**np.random.uniform(-4, -3)
    hidden_dim = int(np.random.choice([32, 64, 128, 256, 512, 1024]))
    dropout = np.random.rand() * 0.9 + 0.1

    params = {
        'hidden_dim': hidden_dim,
        'dropout': dropout,
        'input_dim': embeddings.vector_size
    }

    model = RNNBinaryClassifier(**params)
    params['lr'] = lr
    logger.info('Parameters: %s' % params)

    trained_model = \
        train(model,
              train_dataloader,
              val_dataloader,
              epochs=20,
              noise_level=0,
              lr=lr,
              log_every=2,
              comment='hyperparameters_search_random',
              save_model_path=None)
    metrics = get_metrics(trained_model, val_dataloader)

    if metrics['f1'] > best_f1:
        logger.info('YES!, f1: %s, parameters: %s' % (metrics['f1'], str(params)))
        best_f1 = metrics['f1']
        best_model = model
        best_params = params
        logger.info(best_params)

07-11 19:43 root         INFO     Parameters: {'hidden_dim': 128, 'dropout': 0.7517732019887394, 'input_dim': 300, 'lr': 0.0002307763213920885}
07-11 19:43 root         INFO     Writer: runs/Jul11_19-43-31_lyalin_RNNBinaryClassifier_lr3_dropout0.7517732019887394_noise_level0.0000hyperparameters_search_random
07-11 19:44 root         INFO     Epoch 0. Global step 665. T=0.79min
07-11 19:44 root         INFO     In-batch loss      : 0.7073
07-11 19:44 root         INFO     Training accuracy  : 0.5188, f1: 0.6450
07-11 19:44 root         INFO     Validation accuracy: 0.5173, f1: 0.6438
07-11 19:45 root         INFO     Epoch 2. Global step 1995. T=2.33min
07-11 19:45 root         INFO     In-batch loss      : 1.1621
07-11 19:45 root         INFO     Training accuracy  : 0.6085, f1: 0.4900
07-11 19:45 root         INFO     Validation accuracy: 0.6216, f1: 0.5109
07-11 19:47 root         INFO     Epoch 4. Global step 3325. T=3.89min
07-11 19:47 root         INFO     In-batch loss      : 0.1

KeyboardInterrupt: 

In [15]:
best_f1, best_params

(0.8744982606368745,
 {'hidden_dim': 256,
  'dropout': 0.19682083244247645,
  'input_dim': 300,
  'lr': 0.0006258490940383969})

In [ ]:
params ={'hidden_dim': 128,
         'dropout': 0.7517732019887394,
         'input_dim': 300}

model = RNNBinaryClassifier(**params)
params['lr'] = lr = 0.0002307763213920885
logger.info('Parameters: %s' % params)

trained_model = \
    train(model,
          train_dataloader,
          val_dataloader,
          epochs=20,
          noise_level=0,
          lr=lr,
          log_every=2,
          comment='_hyperparameters_search_manual',
          save_model_path=None,
          use_annealing=False)

trained_model.eval()
metrics = get_metrics(trained_model, val_dataloader)

07-13 16:40 root         INFO     Parameters: {'hidden_dim': 128, 'dropout': 0.7517732019887394, 'input_dim': 300, 'lr': 0.0002307763213920885}
07-13 16:40 root         INFO     Writer: runs/Jul13_16-40-11_lyalin_RNNBinaryClassifier_lr3_dropout0.7517732019887394_noise_level0.0000_hyperparameters_search_manual
07-13 16:41 root         INFO     Epoch 0. Global step 665. T=1.16min
07-13 16:41 root         INFO     In-batch loss      : 0.6244
07-13 16:41 root         INFO     Training accuracy  : 0.5200, f1: 0.3444
07-13 16:41 root         INFO     Validation accuracy: 0.5080, f1: 0.3547
07-13 16:44 root         INFO     Epoch 2. Global step 1995. T=3.95min
07-13 16:44 root         INFO     In-batch loss      : 0.6833
07-13 16:44 root         INFO     Training accuracy  : 0.5220, f1: 0.3564
07-13 16:44 root         INFO     Validation accuracy: 0.5157, f1: 0.3157
07-13 16:46 root         INFO     Epoch 4. Global step 3325. T=6.71min
07-13 16:46 root         INFO     In-batch loss      : 0.